### Nonlinear Finite-Difference Method
This notebook implements the nonlinear finite-difference method to approximate the solution to nonlinear boundary value problems of the form
$$
y'' = f(x,y,y'), \; a \leq x \leq b, \; y(a) = \alpha, \; y(b) = \beta.
$$
To use this algorithm, we require the computation of partial derivatives $f_y(x,y,y')$ and $f_{y'}(x,y,y')$. This algorithm is implemented according to the pseudocode given in Algorithm 11.4 of *Numerical Analysis* (10th Edition) by Burden and Faires. The code below provides the solution for 11.4.3(b) in the same book.

*Note*: A maximum number of iterations to obtain convergence is required. For convenience, this is just $N+1$.

In [67]:
# Imports
import numpy as np
import pandas as pd
import math

# For more decimal places
pd.set_option("display.precision", 8)

In [68]:
# Function (and partials)
f = lambda x, y, yp: yp*np.cos(x) - y*np.log(y)
fy = lambda x, y, yp: -np.log(y) - 1
fyp = lambda x, y, yp: np.cos(x)

# Actual solution
y = lambda x: math.exp(np.sin(x))

# Left endpoint
a = 0
# Right endpoint
b = math.pi/2
# Left endpoint value
alpha = 1 
# Right endpoint value
beta = math.exp(1)
# N value
N = 9
# Maximum iterations (for convenience, choose N+1)
M = N+1
# Tolerance
TOL = 10**(-4)

# Initializing arrays (used in approximation computation)
arr_a = np.zeros(N+2)
arr_b = np.zeros(N+2)
arr_c = np.zeros(N+2)
arr_d = np.zeros(N+2)
arr_l = np.zeros(N+2)
arr_u = np.zeros(N+2)
arr_z = np.zeros(N+2)
arr_v = np.zeros(N+2)
w     = np.zeros(N+2)

In [69]:
# Defining l∞ norm
def norm(x):
  return max(abs(x))

In [70]:
# Step 1
h = (b-a)/(N+1)
w[0] = alpha
w[-1] = beta

# Timesteps
x_steps = np.arange(a, b+h, h)

# Step 2
for i in range(1, N+1):
  w[i] = alpha + i*((beta-alpha)/(b-a))*h

In [71]:
# Step 3
k = 1

# Step 4
while k <= M:

  # Step 5
  x = a + h
  t = (w[2] - alpha)/(2*h)
  arr_a[1] = 2 + h**2*fy(x, w[1], t)
  arr_b[1] = -1 + (h/2)*fyp(x, w[1], t)
  arr_d[1] = -(2*w[1] - w[2] - alpha + h**2*f(x, w[1], t))

  # Step 6
  for i in range(2, N):
    x = a + i*h
    t = (w[i+1] - w[i-1])/(2*h)
    arr_a[i] = 2 + h**2*fy(x, w[i], t)
    arr_b[i] = -1 + (h/2)*fyp(x, w[i], t)
    arr_c[i] = -1 - (h/2)*fyp(x, w[i], t)
    arr_d[i] = -(2*w[i] - w[i+1] - w[i-1] + h**2*f(x, w[i], t))

  # Step 7
  x = b - h
  t = (beta - w[-3])/(2*h)
  arr_a[-2] = 2 + h**2*fy(x, w[-2], t)
  arr_c[-2] = -1 - (h/2)*fyp(x, w[-2], t)
  arr_d[-2] = -(2*w[-2] - w[-3] - beta + h**2*f(x, w[-2], t))

  # Step 8
  arr_l[1] = arr_a[1]
  arr_u[1] = arr_b[1]/arr_a[1]
  arr_z[1] = arr_d[1]/arr_l[1]

  # Step 9
  for i in range(2, N):
    arr_l[i] = arr_a[i] - arr_c[i]*arr_u[i-1]
    arr_u[i] = arr_b[i]/arr_l[i]
    arr_z[i] = (arr_d[i] - arr_c[i]*arr_z[i-1])/arr_l[i]

  # Step 10
  arr_l[-2] = arr_a[-2] - arr_c[-2]*arr_u[-3]
  arr_z[-2] = (arr_d[-2] - arr_c[-2]*arr_z[-3])/arr_l[-2]

  # Step 11
  arr_v[-2] = arr_z[-2]
  w[-2] = w[-2] + arr_v[-2]

  # Step 12
  for i in range(N-1, 0, -1):
    arr_v[i] = arr_z[i] - arr_u[i]*arr_v[i+1]
    w[i] = w[i] + arr_v[i]

  # Step 13
  if norm(arr_v) <= TOL:
    # Step 14
    print(f'The procedure was successful after {k} iterations.')
    # Step 15 (output given below)
    break

  # Step 16
  k = k + 1

# Step 17
if k == M:
  print('Maximum number of iterations exceeded.\nThe procedure was not successful.')

The procedure was successful after 3 iterations.


In [72]:
yt = np.zeros(N+2)

# Evaluating true values of function
for idx, val in enumerate(x_steps):
  yt[idx] = y(val)

In [73]:
df = pd.DataFrame({'xᵢ': x_steps, 'wᵢ': w, 'y(xᵢ)': yt})
df

,xᵢ,wᵢ,y(xᵢ)
0,0.00000000,1.00000000,1.00000000
1,0.15707963,1.16942058,1.16933413
2,0.31415927,1.36244080,1.36208552
3,0.47123890,1.57538813,1.57458304
4,0.62831853,1.80138559,1.79999746
5,0.78539816,2.03011674,2.02811498
6,0.94247780,2.24819259,2.24569937
7,1.09955743,2.44026338,2.43758190
8,1.25663706,2.59083695,2.58844295
9,1.41371669,2.68653080,2.68502044
